In [31]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
from shapely.geometry import Polygon
import folium

def part_of_journey(hour):
  if hour <= datetime.time(6, 30):
      return '1 - Avant 6h30'
  if datetime.time(6, 30) < hour <= datetime.time(9, 30):
      return '2 - AM (6h30 à 9h30)'
  if datetime.time(9, 30) < hour <= datetime.time(15, 30):
      return '3 - Jour (9h30 à 15h30)'
  if datetime.time(15, 30) < hour <= datetime.time(18, 30):
      return '4 - PM (15h30 à 18h30)'
  if datetime.time(18, 30) < hour :
    return '5 - Soir (après 18h30)'
  else:
    return 'ERROR'

def part_of_journey_stm(period):
  if period == 'Avant 6h30':
      return '1 - Avant 6h30'
  if period == 'AM (6h30 à 9h30)':
      return '2 - AM (6h30 à 9h30)'
  if period == 'Jour (9h30 à 15h30)':
      return '3 - Jour (9h30 à 15h30)'
  if period == 'PM (15h30 à 18h30)':
      return '4 - PM (15h30 à 18h30)'
  if period == 'Soir (après 18h30)':
    return '5 - Soir (après 18h30)'
  else:
    return 'ERROR'


### Creation du polygon pour la plaza

In [ ]:
coin_rosemont_boyer= [45.53414890262374, -73.59569524098734]
coin_rosemont_st_vallier = [45.53181801496085, -73.59784906811936]
coin_jean_talon_st_vallier = [45.53915366061581, -73.61386735169552]
coin_jean_talon_boyer = [45.54174620305895, -73.61260342943066]

lat_point_list = [45.53414890262374, 45.53181801496085, 45.53915366061581, 45.54174620305895, 45.53414890262374]
lon_point_list = [-73.59569524098734, -73.59784906811936, -73.61386735169552, -73.61260342943066, -73.59569524098734]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = 'epsg:4326'
polygon_plaza = gpd.GeoDataFrame( crs=crs, geometry=[polygon_geom])  


# map = folium.Map([45.53619845067009, -73.60313339047141], zoom_start=15, tiles='cartodbpositron')
# folium.GeoJson(polygon_plaza).add_to(m)
# folium.LatLngPopup().add_to(m)
# map

### Importation des données

In [33]:
communauto_df = pd.read_excel('data/Communauto/Données brutes Communauto.xlsx')
communauto_gdf = gpd.GeoDataFrame(communauto_df, geometry=gpd.points_from_xy(communauto_df.ArretLongitude, communauto_df.ArretLatitude))
communauto_gdf.crs = 'epsg:4326'
# display(communauto_gdf)

bixi_station_df = pd.read_csv('data/Bixi/2021_stations.csv')
bixi_station_gdf = gpd.GeoDataFrame(bixi_station_df, geometry=gpd.points_from_xy(bixi_station_df.longitude, bixi_station_df.latitude))
bixi_station_gdf.crs = 'epsg:4326'

bixi_data_df = pd.read_csv('data/Bixi/2021_donnees_ouvertes.csv')

bixi_merged_df_1 = pd.merge(bixi_data_df,bixi_station_gdf, 
                     left_on='emplacement_pk_start',
                     right_on=bixi_station_gdf['pk'],
                     how='left')

bixi_merged_df_1 = bixi_merged_df_1.fillna(0)
bixi_merged_df = pd.merge(bixi_merged_df_1, bixi_station_gdf,
                     left_on='emplacement_pk_end',
                     right_on=bixi_station_gdf['pk'],
                     how='left',
                      suffixes=('_start', '_end'))
bixi_merged_df.drop(columns=['latitude_start', 'longitude_start','latitude_end', 'longitude_end'], inplace = True)
bixi_merged_gdf = gpd.GeoDataFrame(bixi_merged_df)
# display(bixi_merged_gdf)

stm_df = pd.read_excel('data/STM/Résultat_Projet FabMob Plaza St-H - Liste des arrêts.xlsx', sheet_name='Données')
stm_gdf = gpd.GeoDataFrame(stm_df, geometry=gpd.points_from_xy(stm_df.X, stm_df.Y))
stm_gdf.crs = 'epsg:2950'
stm_gdf = stm_gdf.to_crs(4326)





### Selection des données pour l'experimentation
#### Bixi: 
##### - Selection des données du 1er au 31 octobre 2021
##### - Prise en compte des points d'arrivee seulement
##### - Gestion des periodes de la journée du stop

#### Communauto:
##### - Selection des trajets qui font un arret dans la quartier de la plaza: Type d'arret = Arret ou Nom de station = Flex Montréal
##### - prise en compte des dates et du point de géolocalisation de TripEventDate
##### - Gestion des periodes de la journée du stop

### MTL Trajet
#### Les données n'ont pas ete integrées car elles sont aggrégées pôur H2S donc trop large pour notre cas d'usage

### Parco
#### En attente des données 

In [34]:
communauto_gdf['datDateDebutReservation'] = pd.to_datetime(communauto_gdf['datDateDebutReservation'], format='%d/%m/%y %H:%M')
communauto_gdf['datDateFinReservation'] = pd.to_datetime(communauto_gdf['datDateFinReservation'], format='%d/%m/%y %H:%M')
communauto_gdf['TripEventDate'] = pd.to_datetime(communauto_gdf['TripEventDate'], format='%d/%m/%y %H:%M')
bixi_merged_gdf['end_date'] = pd.to_datetime(bixi_merged_gdf['end_date'], format='%Y-%m-%d %H:%M:%S.%f')


bixi_extract = bixi_merged_gdf[(bixi_merged_gdf['end_date']<= '2021-10-31')& (bixi_merged_gdf['end_date']>= '2021-10-01')].copy()
communauto_gdf['periode'] = communauto_gdf.apply(lambda trip: part_of_journey(trip["TripEventDate"].time()),axis=1)
bixi_extract['periode'] = bixi_extract.apply(lambda trip: part_of_journey(trip["end_date"].time()),axis=1)
stm_gdf['Période'] = stm_gdf.apply(lambda trip: part_of_journey_stm(trip["Période"]),axis=1)



bixi_extract.set_geometry("geometry_end", inplace=True)
bixi_extract_2 = bixi_extract.sjoin(polygon_plaza, how="inner", predicate='within')
display(bixi_extract_2)

bixi_data = bixi_extract_2[['periode', 'end_date', 'emplacement_pk_end', 'name_end','is_member', 'geometry_end']].copy()
bixi_data["Type d'arrêt"] = 'Fin de trajet'	
bixi_data["Mode"] = 'Bixi'	
# bixi_data.drop(columns='duration_sec', inplace=True)
bixi_data = bixi_data.rename({'end_date': 'date_stop', 'emplacement_pk_end': 'id_station', 'name_end': 'nom_station', "Type d'arrêt": "type_arret", "geometry_end": 'geometry'}, axis=1)

communauto_extract= communauto_gdf[(communauto_gdf["Type d'arrêt"]=='Arrêt') | (communauto_gdf["strNomStation"]=='FLEX Montréal')].copy()
communauto_data = communauto_extract[['periode', 'TripEventDate', 'StationNo', 'strNomStation','Type d\'arrêt','geometry']].copy()

communauto_data['is_member'] =1
communauto_data["Mode"] = 'Communauto'	
communauto_data.reset_index(inplace=True)
# communauto_data.drop(columns='TripEventDate_idx', inplace=True)
communauto_data = communauto_data.rename({'TripEventDate': 'date_stop', 'StationNo': 'id_station', 'strNomStation': 'nom_station', "Type d'arrêt": "type_arret"}, axis=1)

# rdf = gpd.GeoDataFrame( pd.concat( dataframesList, ignore_index=True) )

data_plaza =gpd.GeoDataFrame( pd.concat([bixi_data, communauto_data], ignore_index=True))
data_plaza['day_of_week'] = data_plaza['date_stop'].dt.day_name()
data_plaza['IsWeekend'] = data_plaza['date_stop'].dt.weekday >= 5

data_plaza['date'] = data_plaza['date_stop'].dt.date
display(data_plaza)
data_plaza_json =data_plaza.copy()
data_plaza_json['date_stop'] = data_plaza_json['date_stop'].astype(str)
data_plaza_json['date'] = data_plaza_json['date'].astype(str)


,start_date,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member,pk_start,name_start,geometry_start,pk_end,name_end,geometry_end,periode,index_right
12800,2021-10-01 12:21:51.862,17,2021-10-01 12:55:08.310,592,1996,1,17,Tolhurst / Fleury,POINT (-73.66752 45.54415),592,Boyer / St-Zotique,POINT (-73.60556 45.53848),3 - Jour (9h30 à 15h30),0
13331,2021-10-07 09:31:12.345,17,2021-10-07 10:10:33.117,551,2360,1,17,Tolhurst / Fleury,POINT (-73.66752 45.54415),551,Métro Beaubien (de Chateaubriand / Beaubien),POINT (-73.60363 45.53536),3 - Jour (9h30 à 15h30),0
13334,2021-10-07 09:31:08.416,17,2021-10-07 10:10:34.215,551,2365,1,17,Tolhurst / Fleury,POINT (-73.66752 45.54415),551,Métro Beaubien (de Chateaubriand / Beaubien),POINT (-73.60363 45.53536),3 - Jour (9h30 à 15h30),0
13823,2021-10-05 11:44:22.975,17,2021-10-05 12:05:24.541,592,1261,1,17,Tolhurst / Fleury,POINT (-73.66752 45.54415),592,Boyer / St-Zotique,POINT (-73.60556 45.53848),3 - Jour (9h30 à 15h30),0
15006,2021-10-12 18:52:15.024,17,2021-10-12 19:19:56.280,569,1661,0,17,Tolhurst / Fleury,POINT (-73.66752 45.54415),569,Boyer / Bélanger,POINT (-73.60897 45.54006),5 - Soir (après 18h30),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564999,2021-10-26 19:39:50.087,1133,2021-10-26 19:44:50.882,569,300,1,1133,Parc Turin (Chambord / Jean-Talon),POINT (-73.61016 45.54468),569,Boyer / Bélanger,POINT (-73.60897 45.54006),5 - Soir (après 18h30),0
5565002,2021-10-28 20:19:24.054,1133,2021-10-28 20:23:27.636,569,243,1,1133,Parc Turin (Chambord / Jean-Talon),POINT (-73.61016 45.54468),569,Boyer / Bélanger,POINT (-73.60897 45.54006),5 - Soir (après 18h30),0
5565051,2021-10-26 09:06:17.626,1133,2021-10-26 09:11:51.280,593,333,1,1133,Parc Turin (Chambord / Jean-Talon),POINT (-73.61016 45.54468),593,de St-Vallier / St-Zotique,POINT (-73.60755 45.53626),2 - AM (6h30 à 9h30),0
5565057,2021-10-26 09:37:04.630,1133,2021-10-26 09:41:29.467,593,264,1,1133,Parc Turin (Chambord / Jean-Talon),POINT (-73.61016 45.54468),593,de St-Vallier / St-Zotique,POINT (-73.60755 45.53626),3 - Jour (9h30 à 15h30),0


,periode,date_stop,id_station,nom_station,is_member,geometry,type_arret,Mode,index,day_of_week,IsWeekend,date
0,3 - Jour (9h30 à 15h30),2021-10-01 12:55:08.310,592,Boyer / St-Zotique,1,POINT (-73.60556 45.53848),Fin de trajet,Bixi,NaN,Friday,False,2021-10-01
1,3 - Jour (9h30 à 15h30),2021-10-07 10:10:33.117,551,Métro Beaubien (de Chateaubriand / Beaubien),1,POINT (-73.60363 45.53536),Fin de trajet,Bixi,NaN,Thursday,False,2021-10-07
2,3 - Jour (9h30 à 15h30),2021-10-07 10:10:34.215,551,Métro Beaubien (de Chateaubriand / Beaubien),1,POINT (-73.60363 45.53536),Fin de trajet,Bixi,NaN,Thursday,False,2021-10-07
3,3 - Jour (9h30 à 15h30),2021-10-05 12:05:24.541,592,Boyer / St-Zotique,1,POINT (-73.60556 45.53848),Fin de trajet,Bixi,NaN,Tuesday,False,2021-10-05
4,5 - Soir (après 18h30),2021-10-12 19:19:56.280,569,Boyer / Bélanger,0,POINT (-73.60897 45.54006),Fin de trajet,Bixi,NaN,Tuesday,False,2021-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...
15662,5 - Soir (après 18h30),2021-10-09 18:33:35.000,575,St-Joseph O. et Jeanne-Mance,1,POINT (-73.59536 45.53482),Arrêt,Communauto,7379.0,Saturday,True,2021-10-09
15663,4 - PM (15h30 à 18h30),2021-10-20 17:32:08.000,0,FLEX Montréal,1,POINT (-73.59536 45.53482),Arrêt,Communauto,7380.0,Wednesday,False,2021-10-20
15664,5 - Soir (après 18h30),2021-10-25 23:32:35.000,0,FLEX Montréal,1,POINT (-73.59534 45.53482),Arrêt,Communauto,7381.0,Monday,False,2021-10-25
15665,5 - Soir (après 18h30),2021-10-20 20:01:01.000,340,Henri-Julien et l'ENAP,1,POINT (-73.59531 45.53476),Arrêt,Communauto,7382.0,Wednesday,False,2021-10-20


In [35]:
data_plaza_json.to_file("test.geojson", driver='GeoJSON')

In [36]:
data_plaza_agregat= data_plaza.groupby(['date','periode', 'nom_station','Mode']).agg({'id_station':'count', 'IsWeekend': 'first', 'geometry': 'first'}).reset_index()
display(data_plaza_agregat)
data_plaza_agregat.to_excel("data.xlsx")

,date,periode,nom_station,Mode,id_station,IsWeekend,geometry
0,2021-10-01,1 - Avant 6h30,Boyer / Bélanger,Bixi,5,False,POINT (-73.60897 45.54006)
1,2021-10-01,1 - Avant 6h30,Boyer / Jean-Talon,Bixi,3,False,POINT (-73.61248 45.54159)
2,2021-10-01,1 - Avant 6h30,Boyer / St-Zotique,Bixi,3,False,POINT (-73.60556 45.53848)
3,2021-10-01,1 - Avant 6h30,FLEX Montréal,Communauto,1,False,POINT (-73.59771 45.53371)
4,2021-10-01,1 - Avant 6h30,Métro Beaubien (de Chateaubriand / Beaubien),Bixi,6,False,POINT (-73.60363 45.53536)
...,...,...,...,...,...,...,...
2408,2021-11-01,3 - Jour (9h30 à 15h30),De l'Esplanade et Beaubien,Communauto,1,False,POINT (-73.60912 45.53688)
2409,2021-11-01,4 - PM (15h30 à 18h30),De Chateaubriand et Beaubien,Communauto,2,False,POINT (-73.60248 45.53628)
2410,2021-11-01,4 - PM (15h30 à 18h30),FLEX Montréal,Communauto,1,False,POINT (-73.61158 45.53802)
2411,2021-11-01,4 - PM (15h30 à 18h30),Rosemont et Christophe-Colomb,Communauto,1,False,POINT (-73.59843 45.53535)


In [37]:

stm_gdf_ready = stm_gdf[['Période', 'Description de l\'arrêt', 'Descendants', 'geometry']].copy()
stm_gdf_ready['IsWeekend']= 'False'
stm_gdf_ready['Mode']= 'Bus'
stm_gdf_ready['Descendants'] = stm_gdf_ready['Descendants']*-1
stm_gdf_ready = stm_gdf_ready.rename({'Période': 'periode','Description de l\'arrêt': 'nom_station','Descendants':'Nb_personnes'}, axis=1)
# display(stm_gdf_ready)

data_plaza_agregat_all = data_plaza_agregat.groupby(['IsWeekend','periode', 'nom_station','Mode']).agg({'id_station':'mean', 'geometry': 'first'}).reset_index()
data_plaza_agregat_all = data_plaza_agregat_all.rename({'id_station': 'Nb_personnes'}, axis=1)
data_plaza_agregat_all.to_excel("data_agregat_all.xlsx")

data_plaza_agregat_week = data_plaza_agregat_all[data_plaza_agregat_all['IsWeekend']==False]
# display(data_plaza_agregat_week)

data_plaza_agregat_week_gdf = gpd.GeoDataFrame(data_plaza_agregat_week, geometry=data_plaza_agregat_week['geometry'])
data_plaza_agregat_week_gdf =gpd.GeoDataFrame( pd.concat([data_plaza_agregat_week_gdf, stm_gdf_ready], ignore_index=True))

data_plaza_agregat_week_gdf['lon'] = data_plaza_agregat_week_gdf['geometry'].x
data_plaza_agregat_week_gdf['lat'] = data_plaza_agregat_week_gdf['geometry'].y
display(data_plaza_agregat_week_gdf)
data_plaza_agregat_week_gdf.to_file("data_plaza_agregat_week.geojson", driver='GeoJSON')
data_plaza_agregat_week_gdf.to_csv("data_plaza_agregat_week.csv")

,IsWeekend,periode,nom_station,Mode,Nb_personnes,geometry,lon,lat
0,False,1 - Avant 6h30,Boyer / Bélanger,Bixi,2.421053,POINT (-73.60897 45.54006),-73.608970,45.540060
1,False,1 - Avant 6h30,Boyer / Jean-Talon,Bixi,2.000000,POINT (-73.61248 45.54159),-73.612481,45.541586
2,False,1 - Avant 6h30,Boyer / St-Zotique,Bixi,2.421053,POINT (-73.60556 45.53848),-73.605560,45.538480
3,False,1 - Avant 6h30,Boyer et des Carrières (P sur rue),Communauto,1.000000,POINT (-73.59564 45.53455),-73.595643,45.534547
4,False,1 - Avant 6h30,Cartier et Rosemont,Communauto,1.000000,POINT (-73.60246 45.53691),-73.602456,45.536907
...,...,...,...,...,...,...,...,...
538,False,5 - Soir (après 18h30),Rosemont / Saint-Hubert,Bus,10.000000,POINT (-73.59656 45.53329),-73.596560,45.533291
539,False,5 - Soir (après 18h30),Rosemont / Saint-Hubert,Bus,44.000000,POINT (-73.59656 45.53329),-73.596560,45.533291
540,False,5 - Soir (après 18h30),Rosemont / Saint-Hubert,Bus,6.000000,POINT (-73.59656 45.53305),-73.596560,45.533049
541,False,5 - Soir (après 18h30),Rosemont / Saint-Hubert,Bus,14.000000,POINT (-73.59656 45.53305),-73.596560,45.533049
